## Import 및 한글 설정

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import unicodedata
import random
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models, utils
from torch.utils.data import DataLoader

In [ ]:
plt.rc('font', family='NanumGothic')
plt.rc('font', size=15)

## Mission 1-1
- 각 메뉴를 클래스로 하는 분류 데이터셋과 데이터로더를 준비
- 예시 이미지를 클래스별로 하나씩 총 42장을 한번에 시각화(plotting)하여 확인

### Data resize (224*224)
- 시각화되는 이미지의 크기를 일정하게 조절
- 이후 학습에 사용될 때에도 같은 크기의 이미지를 사용

In [ ]:
def resize_images(input_dir, output_dir, target_size=(224, 224)):
    for class_name in os.listdir(input_dir):
        print(f"생성중인 class : {class_name}")
        class_dir = os.path.join(input_dir, class_name)
        if os.path.isdir(class_dir):
            output_class_dir = os.path.join(output_dir, class_name)
            os.makedirs(output_class_dir, exist_ok=True)

            file_list = os.listdir(class_dir)

            for i, file_name in enumerate(tqdm(file_list, desc=f"Resizing images in {class_name}"), start=1):
                file_path = os.path.join(class_dir, file_name)
                image = Image.open(file_path)
                image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
                resized_image = cv2.resize(image, target_size)
                output_file_path = os.path.join(output_class_dir, file_name)
                cv2.imwrite(output_file_path, resized_image)

In [ ]:
# 훈련 데이터 resize
input_directory = "/content/drive/MyDrive/kfood_train/train"
output_directory = "/content/drive/MyDrive/kfood_train_resized/train"

resize_images(input_directory, output_directory)

생성중인 class : 갈치구이


Resizing images in 갈치구이: 100%|██████████| 809/809 [00:15<00:00, 53.72it/s] 


생성중인 class : 갈비구이


Resizing images in 갈비구이: 100%|██████████| 810/810 [00:14<00:00, 54.83it/s]


생성중인 class : 감자전


...

Resizing images in 파전: 100%|██████████| 797/797 [00:17<00:00, 46.87it/s] 


생성중인 class : 황태구이


Resizing images in 황태구이: 100%|██████████| 805/805 [00:14<00:00, 55.52it/s] 


생성중인 class : 훈제오리


Resizing images in 훈제오리: 100%|██████████| 796/796 [00:13<00:00, 60.34it/s] 


In [ ]:
# 검증 데이터 resize
input_directory = "/content/drive/MyDrive/kfood_val/val"
output_directory = "/content/drive/MyDrive/kfood_val_resized/val"

resize_images(input_directory, output_directory)

생성중인 class : 갈비구이


Resizing images in 갈비구이: 100%|██████████| 85/85 [00:24<00:00,  3.44it/s]


생성중인 class : 계란국


Resizing images in 계란국: 100%|██████████| 97/97 [00:27<00:00,  3.57it/s]


생성중인 class : 경단


...

Resizing images in 호박전: 100%|██████████| 85/85 [00:26<00:00,  3.24it/s]


생성중인 class : 파전


Resizing images in 파전: 100%|██████████| 103/103 [00:02<00:00, 41.62it/s]


생성중인 class : 훈제오리


Resizing images in 훈제오리: 100%|██████████| 113/113 [00:01<00:00, 62.29it/s] 


### device 및 transform 설정

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

### 데이터 로드

In [ ]:
root_dir_train = '/content/drive/MyDrive/kfood_train_resized/train'
root_dir_val = '/content/drive/MyDrive/kfood_val_resized/val'

In [ ]:
# 데이터셋 로드
train_dataset = datasets.ImageFolder(root=root_dir_train, transform=transform)
val_dataset = datasets.ImageFolder(root=root_dir_val, transform=transform)

# 데이터로더 설정
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


### 클래스별 이미지 시각화
- os.listdir : 디렉토리의 이름을 리스트로 반환하는 함수
- os.path.join : 파일 또는 디렉토리의 이름을 합쳐 경로를 반환하는 함수

In [ ]:
def plot_random_images(root_dir_train, rows=7, cols=6, figsize=(30, 30)):
    # 7 x 6 subplot 생성
    fig, axes = plt.subplots(rows, cols, figsize=figsize)

    # 데이터 디렉토리 설정
    class_names = os.listdir(root_dir_train)

    # 유니코드 문자열 정규화
    normalized_class_names = [unicodedata.normalize('NFC', name) for name in class_names]

    # subplot 순회 및 이미지 플로팅
    for ax, class_name in zip(axes.flatten(), class_names):
        class_dir = os.path.join(root_dir_train, class_name)
        image_files = os.listdir(class_dir)

        # 무작위 이미지 선택
        if image_files:
            random_image = random.choice(image_files)
            img_path = os.path.join(class_dir, random_image)

            image = Image.open(img_path)

            # 이미지 플로팅 및 정규화된 클래스 이름으로 title 설정
            ax.imshow(image)
            ax.set_title(normalized_class_names[class_names.index(class_name)])

            # X 및 Y 축 숫자, 이미지 경계선 제거
            ax.set_xticks([])
            ax.set_yticks([])
            ax.axis('off')

    plt.show()

# 이미지 시각화
plot_random_images(root_dir_train)

Output hidden; open in https://colab.research.google.com to view.

## Mission 1-2
- ResNet18를 활용하여 42종의 클래스 분류를 수행하고
Validation 데이터에 대한 정확도를 제시
---
- 조건 1: ResNet18의 parameters는 무작위로 초기화하여 사용
- 조건 2: 학습 길이는 50 epoch를 기본으로 하되 추가해도 가능

### 모델(resnet18) 설정

In [ ]:
# resnet18 모델 생성
model = models.resnet18(pretrained=False)

# 출력 클래스 수 42로 설정
model.fc = nn.Linear(model.fc.in_features, 42)
model = model.to(device)

### 손실 함수, 옵티마이저 설정

In [ ]:
# 손실함수를 CrossEntropy로 설정
criterion = nn.CrossEntropyLoss()

# 최적화 기법으로 Adam 사용 및 학습률은 0.001로 설정
optimizer = optim.Adam(model.parameters(), lr=0.001)

### 학습 함수

In [ ]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader, desc="Training"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(train_loader.dataset)

### 평가 함수

In [ ]:
def evaluate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    corrects = 0
    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc="Validation"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, preds = outputs.max(1)
            corrects += (preds == labels).sum().item()
    return running_loss / len(val_loader.dataset), corrects / len(val_loader.dataset) * 100

### 학습 및 평가
- 성능이 점차 증가하다가 에포크 41에서 최고 정확도 67.8% 달성

In [ ]:
best_acc = 0.0

# 에포크는 50으로 설정
for epoch in range(50):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)

    print(f"Epoch {epoch+1}/{50} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f} - Val Acc: {val_acc:.2f}%")

    if val_acc > best_acc:
        best_acc = val_acc
        best_model_wts = model.state_dict()

        # 체크포인트 저장
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': best_model_wts,
            'optimizer_state_dict': optimizer.state_dict(),
            'best_acc': best_acc,
            'loss': val_loss,
        }, "/content/drive/MyDrive/Mission1.pt")

Validation: 100%|██████████| 66/66 [04:39<00:00,  4.23s/it]


Epoch 1/50 - Train Loss: 2.9064 - Val Loss: 2.5024 - Val Acc: 28.01%


Validation: 100%|██████████| 66/66 [00:11<00:00,  5.60it/s]


Epoch 2/50 - Train Loss: 2.3124 - Val Loss: 2.1670 - Val Acc: 36.99%


Validation: 100%|██████████| 66/66 [00:11<00:00,  5.73it/s]


...

Epoch 48/50 - Train Loss: 0.0282 - Val Loss: 2.3654 - Val Acc: 65.01%


Validation: 100%|██████████| 66/66 [00:11<00:00,  5.62it/s]


Epoch 49/50 - Train Loss: 0.0328 - Val Loss: 2.4403 - Val Acc: 64.53%


Validation: 100%|██████████| 66/66 [00:11<00:00,  5.51it/s]

Epoch 50/50 - Train Loss: 0.0513 - Val Loss: 2.2012 - Val Acc: 67.03%
